# Liquid Neural Network

Trained for stock market predictions

In [ ]:
%pip install torch numpy pandas matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# data prep
def load_stock_data(csv_path, sequence_length=60):
    df = pd.read_csv(csv_path)
    data = df["Close"].values
    data = (data - np.mean(data)) / np.std(data)

    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])

    X = np.array(X)
    y = np.array(y)
    
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [ ]:
# Liquid Neural Network
class LiquidNeuron(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LiquidNeuron, self).__init__()
        self.hidden_size = hidden_size
        self.W = nn.Parameter(torch.randn(hidden_size, input_size) * 0.1)
        self.U = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.1)
        self.bias = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x, h_prev):
        # Liquid Time-Constant (LTC) dynamics
        dxdt = torch.tanh(torch.matmul(x, self.W.t()) + torch.matmul(h_prev, self.U.t()) + self.bias)
        h = h_prev + dxdt  # Euler integration
        return h

class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LiquidNeuralNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.liquid_cells = nn.ModuleList([LiquidNeuron(input_size if i == 0 else hidden_size, hidden_size) for i in range(num_layers)])
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h = torch.zeros(x.size(0), self.hidden_size).to(device)
        for t in range(x.size(1)):
            inp = x[:, t]
            for cell in self.liquid_cells:
                h = cell(inp, h)
        out = self.fc(h)
        return out

In [ ]:
# Training
def train_model(model, X_train, y_train, epochs=100, lr=0.001):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        inputs, targets = X_train.to(device), y_train.to(device)

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Prediction Function
def predict(model, X_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test.to(device))
    return predictions.cpu().numpy()


In [ ]:
# Load your CSV file here
X, y = load_stock_data('stock_data.csv')
split_ratio = 0.8
split = int(len(X) * split_ratio)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = LiquidNeuralNetwork(input_size=1, hidden_size=64, output_size=1, num_layers=2)
train_model(model, X_train.unsqueeze(-1), y_train, epochs=100, lr=0.001)

predictions = predict(model, X_test.unsqueeze(-1))

# Plotting Results
plt.figure(figsize=(10, 5))
plt.plot(y_test.numpy(), label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.title('Stock Price Prediction with Liquid Neural Network')
plt.xlabel('Time')
plt.ylabel('Normalized Price')
plt.show()
